In [282]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [283]:
df = pd.read_csv('dados_prova_nivelo.csv')

In [284]:
df.head()

,id_aluno,bncc_questao_1,acertou_questao_1,bncc_questao_2,acertou_questao_2,bncc_questao_3,acertou_questao_3,bncc_questao_4,acertou_questao_4,bncc_questao_5,...,bncc_questao_7,acertou_questao_7,bncc_questao_8,acertou_questao_8,bncc_questao_9,acertou_questao_9,bncc_questao_10,acertou_questao_10,nota_final,dificuldade
0,1,EF06MA03,1,EF07MA11,1,EF08MA12,1,EF09MA06,1,EM13MAT101,...,EF06MA08,1,EF07MA05,0,EF09MA08,1,EM13MAT303,1,9,0
1,2,EF06MA03,0,EF07MA11,1,EF08MA12,0,EF09MA06,1,EM13MAT101,...,EF06MA08,0,EF07MA05,1,EF09MA08,0,EM13MAT303,1,5,1
2,3,EF06MA03,1,EF07MA11,0,EF08MA12,0,EF09MA06,1,EM13MAT101,...,EF06MA08,0,EF07MA05,0,EF09MA08,0,EM13MAT303,0,4,1
3,4,EF06MA03,0,EF07MA11,0,EF08MA12,1,EF09MA06,0,EM13MAT101,...,EF06MA08,0,EF07MA05,0,EF09MA08,1,EM13MAT303,0,3,1
4,5,EF06MA03,1,EF07MA11,1,EF08MA12,0,EF09MA06,0,EM13MAT101,...,EF06MA08,1,EF07MA05,0,EF09MA08,0,EM13MAT303,0,4,1


## Engenharia de variaveis

In [285]:
# Transformando o DataFrame para o formato longo (melted)
bncc_cols = [col for col in df.columns if 'bncc_questao' in col]
acerto_cols = [col for col in df.columns if 'acertou_questao' in col]

# Transformação para um DataFrame long com prefixo BNCC e acerto correspondente
bncc_melted = df.melt(id_vars=['id_aluno'], value_vars=bncc_cols, var_name='questao', value_name='bncc_prefixo')
acerto_melted = df.melt(id_vars=['id_aluno'], value_vars=acerto_cols, var_name='questao', value_name='acertou')

# Removendo o sufixo '_questao_x' das colunas "questao" para combinar as duas tabelas
bncc_melted['questao'] = bncc_melted['questao'].str.replace('bncc_questao_', '')
acerto_melted['questao'] = acerto_melted['questao'].str.replace('acertou_questao_', '')

# Juntando os DataFrames de prefixo e acerto
df_long = pd.merge(bncc_melted, acerto_melted, on=['id_aluno', 'questao'])

# Extraindo o prefixo de cada questão
df_long['bncc_prefixo'] = df_long['bncc_prefixo'].str[:6]

# Calculando a taxa de acerto por prefixo
taxa_acerto_por_prefixo = (
    df_long.groupby(['id_aluno', 'bncc_prefixo'])['acertou']
    .mean()
    .unstack(fill_value=0)
    .add_prefix('taxa_acerto_')
)

# Unindo as taxas de acerto com o DataFrame original
df = df.join(taxa_acerto_por_prefixo, on='id_aluno')


In [286]:
df.shape

(50000, 28)

In [287]:
# Calculando a taxa de acerto por prefixo
acerto_por_prefixo = (
    df_long.groupby(['id_aluno','bncc_prefixo'])['acertou'].sum()
    .unstack(fill_value=0)
    .add_prefix('acertou_')
)


In [288]:
# Unindo as taxas de acerto com o DataFrame original
df = df.join(acerto_por_prefixo, on='id_aluno')

In [289]:
# 2. Total de acertos e erros
acertos_cols = df.filter(like="acertou_questao")
df['total_acertos'] = acertos_cols.sum(axis=1)
df['total_erros'] = (acertos_cols == 0).sum(axis=1)

# 3. Taxa de acerto geral
df['taxa_acerto_geral'] = df['total_acertos'] / acertos_cols.shape[1]

# 4. Dificuldade relativa das questões
df['dificuldade_media_acertos'] = df['dificuldade'] / df['total_acertos'].replace(0, 1)

# 5. Peso de cada acerto na nota final
df['peso_acerto'] = df['nota_final'] / df['total_acertos'].replace(0, 1)

# 6. Agrupamento de questões por dificuldade
# (Supondo que as questões 1, 2 sejam as mais faceis, questoes 3 e 4 nivel medio e 5 e 6 questoes de BNCCs mais dificeis)
df['taxa_acerto_facil'] = df[['acertou_questao_1', 'acertou_questao_2']].mean(axis=1)
df['taxa_acerto_medio'] = df[['acertou_questao_3', 'acertou_questao_4']].mean(axis=1)
df['taxa_acerto_dificil'] = df[['acertou_questao_5', 'acertou_questao_6', 'acertou_questao_7']].mean(axis=1)


# 8. Combinações de acertos por áreas relacionadas
# Por exemplo, combinando questões de matemática EF06MA e EF07MA
df['taxa_acerto_EF06_EF07'] = df[['acertou_questao_1', 'acertou_questao_2', 'acertou_questao_6', 'acertou_questao_7']].mean(axis=1)



In [290]:
df.columns

Index(['id_aluno', 'bncc_questao_1', 'acertou_questao_1', 'bncc_questao_2',
       'acertou_questao_2', 'bncc_questao_3', 'acertou_questao_3',
       'bncc_questao_4', 'acertou_questao_4', 'bncc_questao_5',
       'acertou_questao_5', 'bncc_questao_6', 'acertou_questao_6',
       'bncc_questao_7', 'acertou_questao_7', 'bncc_questao_8',
       'acertou_questao_8', 'bncc_questao_9', 'acertou_questao_9',
       'bncc_questao_10', 'acertou_questao_10', 'nota_final', 'dificuldade',
       'taxa_acerto_EF06MA', 'taxa_acerto_EF07MA', 'taxa_acerto_EF08MA',
       'taxa_acerto_EF09MA', 'taxa_acerto_EM13MA', 'acertou_EF06MA',
       'acertou_EF07MA', 'acertou_EF08MA', 'acertou_EF09MA', 'acertou_EM13MA',
       'total_acertos', 'total_erros', 'taxa_acerto_geral',
       'dificuldade_media_acertos', 'peso_acerto', 'taxa_acerto_facil',
       'taxa_acerto_medio', 'taxa_acerto_dificil', 'taxa_acerto_EF06_EF07'],
      dtype='object')

## Modelo

In [291]:
from sklearn.ensemble import RandomForestClassifier

In [292]:
df['dificuldade'].value_counts()

,count
dificuldade,
1,41446
0,8554


In [294]:
df = pd.get_dummies(df, columns=['bncc_questao_1', 'bncc_questao_2', 'bncc_questao_3',
                                  'bncc_questao_4', 'bncc_questao_5', 'bncc_questao_6',
                                  'bncc_questao_7', 'bncc_questao_8', 'bncc_questao_9',
                                  'bncc_questao_10'])

In [295]:
df.columns

Index(['id_aluno', 'acertou_questao_1', 'acertou_questao_2',
       'acertou_questao_3', 'acertou_questao_4', 'acertou_questao_5',
       'acertou_questao_6', 'acertou_questao_7', 'acertou_questao_8',
       'acertou_questao_9', 'acertou_questao_10', 'nota_final', 'dificuldade',
       'taxa_acerto_EF06MA', 'taxa_acerto_EF07MA', 'taxa_acerto_EF08MA',
       'taxa_acerto_EF09MA', 'taxa_acerto_EM13MA', 'acertou_EF06MA',
       'acertou_EF07MA', 'acertou_EF08MA', 'acertou_EF09MA', 'acertou_EM13MA',
       'total_acertos', 'total_erros', 'taxa_acerto_geral',
       'dificuldade_media_acertos', 'peso_acerto', 'taxa_acerto_facil',
       'taxa_acerto_medio', 'taxa_acerto_dificil', 'taxa_acerto_EF06_EF07',
       'bncc_questao_1_EF06MA03', 'bncc_questao_2_EF07MA11',
       'bncc_questao_3_EF08MA12', 'bncc_questao_4_EF09MA06',
       'bncc_questao_5_EM13MAT101', 'bncc_questao_6_EM13MAT202',
       'bncc_questao_7_EF06MA08', 'bncc_questao_8_EF07MA05',
       'bncc_questao_9_EF09MA08', 'bncc_que

In [299]:
df['idade'] = np.random.randint(5, 25, size=len(df))

In [307]:
df['Quantidade_faltas'] = np.random.randint(0, 50, size=len(df))

In [310]:
df['possui_diagnostico_de_transtorno'] = np.random.choice([0, 1], size=len(df))

In [302]:
df['turma'] = np.random.choice(['EM', 'FM'], size=len(df))

EM = Ensino Médio
FM = Fundamendal

In [304]:
df['turma'] = df['turma'].map({'EM': 0, 'FM': 1})

In [311]:
X =  df[['idade', 'turma','possui_diagnostico_de_transtorno', 'Quantidade_faltas'
          ]]

y = df['dificuldade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = XGBClassifier()
rfc = RandomForestClassifier(random_state=35)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [312]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


y_pred = model.predict(X_test)


# Calculando as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print(f"Accuracy: {accuracy}")
print(f"Precision (Weighted): {precision}")
print(f"Recall (Weighted): {recall}")
print(f"F1 Score (Weighted): {f1}")

Accuracy: 0.829
Precision (Weighted): 0.829
Recall (Weighted): 1.0
F1 Score (Weighted): 0.9065062875888463


## Salvar o modelo

In [259]:
import pickle
import gzip

with gzip.open('model.pkl.gz', 'wb') as f:
    pickle.dump(model, f)

In [316]:
df.to_csv('Dados_tratados.csv', index = False)